<a href="https://colab.research.google.com/github/Amrutha2567/ML/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
# Install YOLOv5 if not already installed
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt

# Import necessary libraries
import torch
import cv2

# Load pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # YOLOv5 small model

# Load video file
video_path = 'path_to_video.mp4'
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Perform detection
    results = model(frame)
    detections = results.xyxy[0]  # (x1, y1, x2, y2, confidence, class)

    # Iterate over detections and draw bounding boxes
    for *box, conf, cls in detections:
        if int(cls) == 0:  # Class 0 is 'person' in COCO dataset
            x1, y1, x2, y2 = map(int, box)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(frame, f'Person {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('YOLOv5 Detection', frame)

    # Break loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Cloning into 'yolov5'...
remote: Enumerating objects: 16953, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 16953 (delta 75), reused 99 (delta 49), pack-reused 16805 (from 1)
Receiving objects: 100% (16953/16953), 15.70 MiB | 21.87 MiB/s, done.
Resolving deltas: 100% (11604/11604), done.
/content/deep_sort/deep_sort/deep_sort/deep_sort/deep_sort/yolov5


Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-9-9 Python-3.10.12 torch-2.4.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 121MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [11]:
pip install scikit-learn filterpy lap


In [12]:
!git clone https://github.com/nwojke/deep_sort.git
%cd deep_sort


Cloning into 'deep_sort'...
remote: Enumerating objects: 141, done.
remote: Total 141 (delta 0), reused 0 (delta 0), pack-reused 141 (from 1)
Receiving objects: 100% (141/141), 65.58 KiB | 1.15 MiB/s, done.
Resolving deltas: 100% (78/78), done.
/content/deep_sort/deep_sort/deep_sort/deep_sort/deep_sort/yolov5/deep_sort


In [17]:
!pip install -r deep_sort/requirements.txt
!python deep_sort/setup.py build_ext --inplace

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'deep_sort/requirements.txt'
python3: can't open file '/content/deep_sort/deep_sort/deep_sort/deep_sort/deep_sort/yolov5/deep_sort/deep_sort/setup.py': [Errno 2] No such file or directory


In [18]:
from deep_sort import DeepSort

# Initialize Deep SORT tracker
deepsort = DeepSort("deep_sort/deep/checkpoint/ckpt.t7") # You may need to provide a path to your model weights

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv5 detection
    results = model(frame)
    detections = results.xyxy[0]  # (x1, y1, x2, y2, confidence, class)

    # Prepare detections for Deep SORT (format: [[x1, y1, x2, y2, confidence, class]])
    bbox_xywh = []
    confidences = []
    for *box, conf, cls in detections:
        if int(cls) == 0:  # Only 'person'
            bbox_xywh.append([box[0], box[1], box[2]-box[0], box[3]-box[1]])  # Convert to (x, y, w, h)
            confidences.append(conf)

    # Pass detections to Deep SORT
    bbox_xywh = torch.Tensor(bbox_xywh)
    confidences = torch.Tensor(confidences)
    outputs = deepsort.update(bbox_xywh, confidences, frame)

    # Draw tracking results (unique IDs) on the frame
    if len(outputs) > 0:
        for output in outputs:
            x1, y1, x2, y2, track_id = output
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame with tracking
    cv2.imshow('Deep SORT Tracking', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

ImportError: cannot import name 'DeepSort' from 'deep_sort' (/content/deep_sort/deep_sort/__init__.py)

In [14]:
from deep_sort.deep_sort import DeepSort

# Initialize Deep SORT tracker
deepsort = DeepSort()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLOv5 detection
    results = model(frame)
    detections = results.xyxy[0]  # (x1, y1, x2, y2, confidence, class)

    # Prepare detections for Deep SORT (format: [[x1, y1, x2, y2, confidence, class]])
    bbox_xywh = []
    confidences = []
    for *box, conf, cls in detections:
        if int(cls) == 0:  # Only 'person'
            bbox_xywh.append([box[0], box[1], box[2]-box[0], box[3]-box[1]])  # Convert to (x, y, w, h)
            confidences.append(conf)

    # Pass detections to Deep SORT
    bbox_xywh = torch.Tensor(bbox_xywh)
    confidences = torch.Tensor(confidences)
    outputs = deepsort.update(bbox_xywh, confidences, frame)

    # Draw tracking results (unique IDs) on the frame
    for output in outputs:
        x1, y1, x2, y2, track_id = output
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f'ID {track_id}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

    # Display the frame with tracking
    cv2.imshow('Deep SORT Tracking', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


ImportError: cannot import name 'DeepSort' from 'deep_sort.deep_sort' (unknown location)

In [20]:
# Video writer initialization
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('https://www.youtube.com/watch?v=JBoc3w5EKfI', fourcc, 20.0, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Your detection and tracking code here

    # Write the frame into the file
    out.write(frame)

cap.release()
out.release()


NameError: name 'frame_width' is not defined